In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import PIL
from PIL import Image

In [3]:
def imshow(img):
    img1 = (img).astype(np.uint8)
    cv2.imshow('Sample',img1)

In [4]:
def pil_imshow(img):
    tyr = Image.fromarray(img)
    tyr.show()

In [5]:
class CannyDetector:
    
    def dx_dy_filters(self,img):
    
        i1 = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5)
        i2 = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5)

        G = np.hypot(i1,i2)
        G = G/G.max()*255
        theta= np.arctan2(i2,i1)

        return G,theta

    def non_max_supression(self,G,theta):
        n,m = G.shape
        supressed_image = np.zeros((n,m),dtype=np.int32)
        theta = theta*180/np.pi
        theta[theta<0]+=180

        for i in range(1,n-1):
            for j in range(1,m-1):
                x = 255
                y = 255
                if (0<=theta[i,j]<22.5) or (157.5<=theta[i,j]<180):
                    x = G[i,j-1]
                    y = G[i,j+1]

                elif (22.5<=theta[i,j]<67.5):
                    x = G[i+1,j-1]
                    y = G[i-1,j+1]

                elif (67.5<=theta[i,j]<112.5):
                    x = G[i+1,j]
                    y = G[i-1,j]

                elif (112.5<=theta[i,j]<157.5) or (157.5<=theta[i,j]<180):
                    x = G[i-1,j-1]
                    y = G[i+1,j+1]

                if G[i,j]>=x and G[i,j]>=y:
                    supressed_image[i,j]=G[i,j]
                else:
                    supressed_image[i,j]=0
        
        return supressed_image
                
    def double_treshold(self,supressed_image,lratio=0.05,hratio=0.09):
        
        hThresh = supressed_image.max()*hratio
        lThresh = hThresh*lratio
        n,m = supressed_image.shape
        after_thresh = np.zeros((n,m),dtype=np.int32)

        weak_value = np.int32(25)    
        strong_value = np.int32(255)

        strong_x,strong_y = np.where(supressed_image>=hThresh)
        weak_x,weak_y = np.where((supressed_image<hThresh) & (supressed_image>=lThresh))

        after_thresh[strong_x,strong_y] = strong_value
        after_thresh[weak_x,weak_y] = weak_value

        return after_thresh,weak_value,strong_value
    
    def hysteresis(self,img,weak_val,strong_val=255):
        n,m = img.shape
        for i in range(1,n-1):
            for j in range(m-1):
                if img[i,j]==weak_val:
                    if (img[i-1,j-1]==strong_val or img[i+1,j+1]==strong_val or img[i+1,j-1]==strong_val or img[i-1,j+1]==strong_val or img[i-1,j]==strong_val or img[i,j-1]==strong_val or img[i,j+1]==strong_val or img[i+1,j]==strong_val):
                        img[i,j]=strong_val
                    else:
                        img[i,j]=0    
    
    def detect_edges(self,img):
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img = cv2.GaussianBlur(img,(5,5),0)
        G,theta = self.dx_dy_filters(img)
        supressed_image = self.non_max_supression(G,theta)
        after_change,weak,strong = self.double_treshold(supressed_image)
        self.hysteresis(after_change,weak,strong)
        return after_change

In [6]:
detector  = CannyDetector()

## Detecting Edges in a live video

In [9]:
vid = cv2.VideoCapture(0)
while True:
    ret,frame = vid.read()
    image_with_edges = detector.detect_edges(frame)
    imshow(image_with_edges)
    key = cv2.waitKey(1)
    if key== ord('q'):
        break
vid.release()
cv2.destroyAllWindows()

## Detecting Edges from an Image 

In [16]:
img2 = cv2.imread('roger.jpg',1)
img2 = cv2.resize(img2,(img2.shape[1]//8,img2.shape[1]//8))
changed_image_visible = detector.detect_edges(img2)
pil_imshow(changed_image_visible)